In [1]:
!pip install matplotlib google-cloud-storage

In [38]:
import collections
import copy
import hashlib
import io
import os
import subprocess
import textwrap
import time
import glob

from typing import List, Text

from PIL import Image

import cv2
import numpy as np
import pandas as pd
import tabulate
import seaborn as sns
from tqdm import tqdm
import ast


import tensorflow as tf

import matplotlib.pyplot as plt

In [3]:
from google.colab import auth

# Authenticate user for access. There will be a popup asking you to sign in with your user and approve access.
auth.authenticate_user()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [77]:
#@title Global params

class Globals:
  # GCP project with GCS bucket of interest
  gcp_project = 'dx-scin-public' #@param

  # GCS bucket with data to read
  gcs_bucket_name = 'dx-scin-public-data' #@param

  # CSV of case metadata to read
  cases_csv = 'dataset/scin_cases.csv' #@param

  # CSV of label metadata to read
  labels_csv = 'dataset/scin_labels.csv' #@param

  # Images directory
  gcs_images_dir = 'dataset/images/' #@param

  ### Key column names
  image_path_columns = ['image_1_path', 'image_2_path', 'image_3_path']
  weighted_skin_condition_label = "weighted_skin_condition_label"
  skin_condition_label = "dermatologist_skin_condition_on_label_name"

  ###### Formed during execution:

  # Client for querying GCS
  gcs_storage_client = None

  # Bucket object for loading files
  gcs_bucket = None

  # pd.DataFrame for the loaded metadata_csv
  cases_df = None

  # pd.DataFrame for the loaded labels_csv
  cases_and_labels_df = None

print(f'GCS bucket name: {Globals.gcs_bucket_name}')
print(f'cases_csv: {Globals.cases_csv}')
print(f'labels_csv: {Globals.labels_csv}')
print(f'images dir: {Globals.gcs_images_dir}')

GCS bucket name: dx-scin-public-data
cases_csv: dataset/scin_cases.csv
labels_csv: dataset/scin_labels.csv
images dir: dataset/images/


## Load dataset

In [78]:
#@title Create a dataframe that contains the metadata and condition labels

from google.cloud import storage

def list_blobs(storage_client, bucket_name):
  """Helper to list blobs in a bucket (useful for debugging)."""
  blobs = storage_client.list_blobs(bucket_name)
  for blob in blobs:
    print(blob)

def initialize_df_with_metadata(bucket, csv_path):
  """Loads the given CSV into a pd.DataFrame."""
  df = pd.read_csv(io.BytesIO(bucket.blob(csv_path).download_as_string()), dtype={'case_id': str})
  df['case_id'] = df['case_id'].astype(str)
  return df

def augment_metadata_with_labels(df, bucket, csv_path):
  """Loads the given CSV into a pd.DataFrame."""
  labels_df = pd.read_csv(io.BytesIO(bucket.blob(csv_path).download_as_string()), dtype={'case_id': str})
  labels_df['case_id'] = labels_df['case_id'].astype(str)
  merged_df = pd.merge(df, labels_df, on='case_id')
  return merged_df

Globals.gcs_storage_client = storage.Client(Globals.gcp_project)
Globals.gcs_bucket = Globals.gcs_storage_client.bucket(
    Globals.gcs_bucket_name
)
Globals.cases_df = initialize_df_with_metadata(Globals.gcs_bucket, Globals.cases_csv)
Globals.cases_and_labels_df = augment_metadata_with_labels(Globals.cases_df, Globals.gcs_bucket, Globals.labels_csv)
print(len(Globals.cases_and_labels_df))

5033


In [7]:
Globals.cases_and_labels_df.columns

Index(['case_id', 'source', 'release', 'year', 'age_group', 'sex_at_birth',
       'fitzpatrick_skin_type',
       'race_ethnicity_american_indian_or_alaska_native',
       'race_ethnicity_asian', 'race_ethnicity_black_or_african_american',
       'race_ethnicity_hispanic_latino_or_spanish_origin',
       'race_ethnicity_middle_eastern_or_north_african',
       'race_ethnicity_native_hawaiian_or_pacific_islander',
       'race_ethnicity_white', 'race_ethnicity_other_race',
       'race_ethnicity_prefer_not_to_answer', 'textures_raised_or_bumpy',
       'textures_flat', 'textures_rough_or_flaky', 'textures_fluid_filled',
       'body_parts_head_or_neck', 'body_parts_arm', 'body_parts_palm',
       'body_parts_back_of_hand', 'body_parts_torso_front',
       'body_parts_torso_back', 'body_parts_genitalia_or_groin',
       'body_parts_buttocks', 'body_parts_leg', 'body_parts_foot_top_or_side',
       'body_parts_foot_sole', 'body_parts_other',
       'condition_symptoms_bothersome_appearan

In [8]:
Globals.cases_and_labels_df.sample(1)

,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us
54,-1180717079993352243,SCIN,1.0.0,2023,AGE_40_TO_49,MALE,FST3,NaN,NaN,NaN,...,NO,NaN,NaN,NaN,NaN,NaN,True,True,2.0,2.0


## Important Functions

**Variables**
* df_original: The full, unmodified dataset containing all cases, metadata, and image paths. This is equivalent to Globals.cases_and_labels_df and can be used like a normal pandas DataFrame.
* df_filtered: A working copy of the dataset that you can safely modify, filter, or clean without affecting the original.
* image_dir : Use this directory to access images within google cloud


**Functions**
* read_image_from_gcs(gcs_path)
  - Downloads and decodes an image directly from your GCS bucket using the path stored in the dataset (e.g. "dataset/images/12345.png").
*  get_all_image_paths(df):
  - Extracts all unique image paths from the three image columns (image_1_path, image_2_path, image_3_path) in the dataset.
* show_case_images(case_id)
  - Displays all available images for a given case_id directly from GCS.
* convert_to_binary_var(col_name)
  - convert to binary values

  Note: add instructions if we want to analyze/change/decode all images together

In [109]:
df_original = Globals.cases_and_labels_df

In [110]:
df_filtered = df_original.copy()

In [111]:
image_dir = "/content/gcs_mount/dataset/images"

In [112]:
def read_image_from_gcs(gcs_path):
    """Reads and decodes an image from GCS (relative path)."""
    try:
        blob = Globals.gcs_bucket.blob(gcs_path)
        img_bytes = blob.download_as_bytes()
        img_array = np.frombuffer(img_bytes, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        if img is None:
            return None
        # Convert BGR (OpenCV) → RGB (matplotlib)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img_rgb
    except Exception as e:
        print(f"Failed to read {gcs_path}: {e}")
        return None


In [113]:
def get_all_image_paths(df):
    """
    Flatten all image_1/2/3 columns from the given DataFrame
    into a unique list of image paths.
    """
    image_cols = ['image_1_path', 'image_2_path', 'image_3_path']

    all_paths = (
        df[image_cols]
        .stack()          # combine all image columns vertically
        .dropna()         # remove missing entries
        .unique()         # only keep unique paths
        .tolist()         # convert to list
    )

    print(f"📸 Found {len(all_paths)} unique image paths.")
    return all_paths


In [114]:
def show_case_images(case_id):
    """Display all available images for a given case_id."""
    row = Globals.cases_and_labels_df[Globals.cases_and_labels_df["case_id"] == str(case_id)]
    if row.empty:
        print(f"No case found for ID {case_id}")
        return

    paths = row[['image_1_path', 'image_2_path', 'image_3_path']].dropna(axis=1).values.flatten()
    plt.figure(figsize=(15, 5))
    for i, path in enumerate(paths, 1):
        img = read_image_from_gcs(path)
        plt.subplot(1, len(paths), i)
        plt.imshow(img)
        plt.title(f"{os.path.basename(path)}", fontsize=9)
        plt.axis("off")
    plt.suptitle(f"Case ID: {case_id}", fontsize=12)
    plt.tight_layout()
    plt.show()


In [115]:
def convert_to_binary_var(col_name):
  df_filtered[col_name].replace('YES', 1, inplace = True)
  df_filtered[col_name].fillna(0, inplace = True)
  print(df_filtered[col_name].value_counts())
  print("")


## No longer need to run this -- Feature Cleaning, Preprocessing, and Exploration

In [86]:
'''
df_filtered['fitzpatrick_skin_type'].fillna('NONE_IDENTIFIED', inplace = True)
print(df_filtered['fitzpatrick_skin_type'].count())
df_filtered['fitzpatrick_skin_type'].value_counts()
'''

5033


/tmp/ipython-input-657253139.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered['fitzpatrick_skin_type'].fillna('NONE_IDENTIFIED', inplace = True)


,count
fitzpatrick_skin_type,
NONE_IDENTIFIED,2858
FST3,661
FST2,542
FST4,427
FST5,214
FST1,188
FST6,143


In [87]:
'''
race_ethnicity_cols_list = ['race_ethnicity_american_indian_or_alaska_native', 'race_ethnicity_asian', 'race_ethnicity_black_or_african_american', 'race_ethnicity_hispanic_latino_or_spanish_origin', 'race_ethnicity_middle_eastern_or_north_african', 'race_ethnicity_native_hawaiian_or_pacific_islander', 'race_ethnicity_white', 'race_ethnicity_other_race', 'race_ethnicity_prefer_not_to_answer', 'race_ethnicity_two_or_more_after_mitigation']

for col in race_ethnicity_cols_list:
  convert_to_binary_var(col)
'''

race_ethnicity_american_indian_or_alaska_native
0.0    4960
1.0      73
Name: count, dtype: int64

race_ethnicity_asian
0.0    4937
1.0      96
Name: count, dtype: int64

race_ethnicity_black_or_african_american
0.0    4735
1.0     298
Name: count, dtype: int64

race_ethnicity_hispanic_latino_or_spanish_origin
0.0    4743
1.0     290
Name: count, dtype: int64

race_ethnicity_middle_eastern_or_north_african
0.0    5026
1.0       7
Name: count, dtype: int64

race_ethnicity_native_hawaiian_or_pacific_islander
0.0    5029
1.0       4
Name: count, dtype: int64

race_ethnicity_white
0.0    3160
1.0    1873
Name: count, dtype: int64

race_ethnicity_other_race
0.0    5017
1.0      16
Name: count, dtype: int64

race_ethnicity_prefer_not_to_answer
0.0    4999
1.0      34
Name: count, dtype: int64

race_ethnicity_two_or_more_after_mitigation
0.0    4950
1.0      83
Name: count, dtype: int64



/tmp/ipython-input-3518719989.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered[col_name].replace('YES', 1, inplace = True)
/tmp/ipython-input-3518719989.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered[col_name].replace('YES', 1, inplace = True)


In [88]:
'''
texture_cols_list = ['textures_raised_or_bumpy', 'textures_flat', 'textures_rough_or_flaky', 'textures_fluid_filled']

for col in texture_cols_list:
  convert_to_binary_var(col)
  '''

textures_raised_or_bumpy
1.0    2915
0.0    2118
Name: count, dtype: int64

textures_flat
0.0    4211
1.0     822
Name: count, dtype: int64

textures_rough_or_flaky
0.0    4001
1.0    1032
Name: count, dtype: int64

textures_fluid_filled
0.0    4404
1.0     629
Name: count, dtype: int64



/tmp/ipython-input-3518719989.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered[col_name].replace('YES', 1, inplace = True)
/tmp/ipython-input-3518719989.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered[col_name].replace('YES', 1, inplace = True)


In [89]:
'''
body_parts_col_start = df_original.columns.get_loc('body_parts_head_or_neck')
body_parts_col_end = df_original.columns.get_loc('body_parts_other')
body_parts_cols_list = df_filtered.iloc[:, body_parts_col_start:body_parts_col_end + 1 ]

for col in body_parts_cols_list:
  convert_to_binary_var(col)
'''

body_parts_head_or_neck
0.0    4296
1.0     737
Name: count, dtype: int64

body_parts_arm
0.0    3326
1.0    1707
Name: count, dtype: int64

body_parts_palm
0.0    4818
1.0     215
Name: count, dtype: int64

body_parts_back_of_hand
0.0    4512
1.0     521
Name: count, dtype: int64

body_parts_torso_front
0.0    4238
1.0     795
Name: count, dtype: int64

body_parts_torso_back
0.0    4381
1.0     652
Name: count, dtype: int64

body_parts_genitalia_or_groin
0.0    4811
1.0     222
Name: count, dtype: int64

body_parts_buttocks
0.0    4642
1.0     391
Name: count, dtype: int64

body_parts_leg
0.0    3339
1.0    1694
Name: count, dtype: int64

body_parts_foot_top_or_side
0.0    4618
1.0     415
Name: count, dtype: int64

body_parts_foot_sole
0.0    4929
1.0     104
Name: count, dtype: int64

body_parts_other
0.0    4490
1.0     543
Name: count, dtype: int64



/tmp/ipython-input-3518719989.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered[col_name].replace('YES', 1, inplace = True)
/tmp/ipython-input-3518719989.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered[col_name].replace('YES', 1, inplace = True)


In [90]:
'''
symptoms_col_start = df_original.columns.get_loc('condition_symptoms_bothersome_appearance')
symptoms_col_end = df_original.columns.get_loc('other_symptoms_no_relevant_symptoms')
symptoms_cols_list = df_filtered.iloc[:, symptoms_col_start:symptoms_col_end + 1 ]

for col in symptoms_cols_list:
  convert_to_binary_var(col)
'''

condition_symptoms_bothersome_appearance
0.0    3503
1.0    1530
Name: count, dtype: int64

condition_symptoms_bleeding
0.0    4808
1.0     225
Name: count, dtype: int64

condition_symptoms_increasing_size
0.0    4020
1.0    1013
Name: count, dtype: int64

condition_symptoms_darkening
0.0    4648
1.0     385
Name: count, dtype: int64

condition_symptoms_itching
1.0    2712
0.0    2321
Name: count, dtype: int64

condition_symptoms_burning
0.0    4045
1.0     988
Name: count, dtype: int64

condition_symptoms_pain
0.0    4299
1.0     734
Name: count, dtype: int64

condition_symptoms_no_relevant_experience
0.0    4725
1.0     308
Name: count, dtype: int64

other_symptoms_fever
0.0    4943
1.0      90
Name: count, dtype: int64

other_symptoms_chills
0.0    4913
1.0     120
Name: count, dtype: int64

other_symptoms_fatigue
0.0    4618
1.0     415
Name: count, dtype: int64

other_symptoms_joint_pain
0.0    4733
1.0     300
Name: count, dtype: int64

other_symptoms_mouth_sores
0.0    4937
1.0 

/tmp/ipython-input-3518719989.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered[col_name].replace('YES', 1, inplace = True)
/tmp/ipython-input-3518719989.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered[col_name].replace('YES', 1, inplace = True)


In [91]:
'''
df_filtered['related_category'].fillna("NO_RESPONSE", inplace=True)
df_filtered['condition_duration'].fillna("NO_RESPONSE", inplace=True)
df_filtered['image_2_shot_type'].fillna("N/A", inplace=True)
df_filtered['image_3_shot_type'].fillna("N/A", inplace=True)
df_filtered['combined_race'].fillna("NO_RESPONSE", inplace=True)
'''

/tmp/ipython-input-3781040459.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered['related_category'].fillna("NO_RESPONSE", inplace=True)
/tmp/ipython-input-3781040459.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [71]:
'''
df_filtered = df_filtered[
    ~(
        (df_filtered['dermatologist_skin_condition_on_label_name'].isna() | (df_filtered['dermatologist_skin_condition_on_label_name'].astype(str) == '[]')) &
        (df_filtered['dermatologist_skin_condition_confidence'].isna() | (df_filtered['dermatologist_skin_condition_confidence'].astype(str) == '[]'))
    )
]
print(len(df_filtered))
'''

3061


In [72]:
'''
print(df_filtered['dermatologist_skin_condition_on_label_name'].value_counts()[:30])
print(df_filtered['dermatologist_skin_condition_confidence'].value_counts()[:30])
'''

dermatologist_skin_condition_on_label_name
[Eczema]                                                              120
[Urticaria]                                                            81
[Eczema, Allergic Contact Dermatitis]                                  67
[Allergic Contact Dermatitis, Irritant Contact Dermatitis]             60
[Allergic Contact Dermatitis]                                          42
[Folliculitis]                                                         37
[Urticaria, Insect Bite, Allergic Contact Dermatitis]                  36
[Acute dermatitis, NOS]                                                27
[Insect Bite]                                                          27
[Tinea, Psoriasis, Eczema]                                             23
[Urticaria, Allergic Contact Dermatitis]                               22
[Psoriasis, Eczema]                                                    21
[O/E - ecchymoses present]                                           

In [98]:
'''
# Convert stringified lists/dicts to real Python objects
for col in [
    "dermatologist_skin_condition_on_label_name",
    "dermatologist_skin_condition_confidence",
    "weighted_skin_condition_label",
]:
    df_filtered[col] = df_filtered[col].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

#  only the aligned condition and confidence lists
df_filtered = df_filtered.explode(
    ["dermatologist_skin_condition_on_label_name", "dermatologist_skin_condition_confidence"]
).reset_index(drop=True)

# Rename for clarity
df_filtered = df_filtered.rename(columns={
    "dermatologist_skin_condition_on_label_name": "condition_name",
    "dermatologist_skin_condition_confidence": "condition_confidence"
})

#  Map the matching weight by condition name
df_filtered["condition_weight"] = df_filtered.apply(
    lambda row: (
        row["weighted_skin_condition_label"].get(row["condition_name"])
        if isinstance(row["weighted_skin_condition_label"], dict)
        else None
    ),
    axis=1
)

# Drop the full dictionary column
df_filtered = df_filtered.drop(columns=["weighted_skin_condition_label"])

# Preview final structure
print(df_filtered[["case_id", "condition_name", "condition_confidence", "condition_weight"]].head(10))
'''

                case_id               condition_name condition_confidence  \
0  -1000600354148496558       Inflicted skin lesions                    4   
1  -1000600354148496558                       Eczema                    4   
2  -1000600354148496558  Irritant Contact Dermatitis                    3   
3  -1002039107727665188            Prurigo nodularis                    4   
4  -1002039107727665188            Actinic Keratosis                    3   
5  -1002039107727665188                    SCC/SCCIS                    4   
6  -1003358831658393077                     Impetigo                    2   
7  -1003358831658393077                Herpes Zoster                    1   
8  -1003358831658393077           Bullous dermatitis                    1   
9  -1003826561155964328                          NaN                  NaN   

   condition_weight  
0              0.41  
1              0.41  
2              0.18  
3              0.41  
4              0.18  
5              0.41 

In [101]:
'''
save_path = os.path.join(save_dir, "dermatologist_conditions_split.csv")
df_filtered.to_csv(save_path, index=False)
'''

 ## Image Cleaning

In [92]:
from google.cloud import storage
# List all images in dataset/images/
all_images = []
for blob in Globals.gcs_bucket.list_blobs(prefix=Globals.gcs_images_dir):
    if blob.name.endswith((".jpg",".jpeg",".png")):
        all_images.append(blob.name)

In [93]:
# Problematic images identified by CleanVision
to_drop = [
    "dataset/images/-4593817128438983108.png",
    "dataset/images/-2431769699504014881.png"
]

# Columns that contain image paths
image_cols = ['image_1_path', 'image_2_path', 'image_3_path']

# Build a boolean mask for rows that have any of these images
mask = df_filtered[image_cols].isin(to_drop).any(axis=1)

# Apply filter (keep only rows without those images)
df_filtered = df_filtered[~mask].copy()

In [94]:
save_dir = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset"
os.makedirs(save_dir, exist_ok=True)

In [95]:
save_path = os.path.join(save_dir, "image_quality_report.csv")
quality_df = pd.read_csv(save_path)
print("Loaded", len(quality_df), "rows")
quality_df.head()

Loaded 10379 rows


,image_path,blur,brightness_mean,brightness_std,underexp,overexp,contrast,shadow
0,dataset/images/-1001492676369731180.png,49.800900,125.822289,62.991703,0.152014,0.049442,0.988142,0.162389
1,dataset/images/-1001733364362669777.png,238.390468,74.623044,29.900723,0.036180,0.000554,1.000000,0.036180
2,dataset/images/-1003800477193786941.png,89.631693,128.779480,47.623139,0.005774,0.023419,0.980237,0.055794
3,dataset/images/-1005922060850163675.png,4.172010,102.138025,62.690025,0.179942,0.000000,0.990610,0.010904
4,dataset/images/-1007969568196430462.png,9.460231,147.597770,40.054561,0.000000,0.002477,0.819608,0.107458


In [96]:
# Problematic images identified by CleanVision
to_drop_2 = [
    "dataset/images/4207723573736028617.png",
]

mask_2 = df_filtered[image_cols].isin(to_drop_2).any(axis=1)
df_filtered = df_filtered.loc[~mask_2]

print(f"Removed {mask_2.sum()} rows containing problematic images.")
print(f" Filtered dataset now has {len(df_filtered)} rows.")

Removed 1 rows containing problematic images.
 Filtered dataset now has 5030 rows.


In [97]:
save_path = os.path.join(save_dir, "image_quality_report_with_sharpness.csv")
quality_df = pd.read_csv(save_path)
print("Loaded", len(quality_df), "rows")
quality_df.head()

Loaded 8209 rows


/tmp/ipython-input-2692060160.py:2: DtypeWarning: Columns (69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  quality_df = pd.read_csv(save_path)


,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us
0,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0
1,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0
2,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0
3,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0
4,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0


# Training Dataset defined


In [106]:
df_filtered_path = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset/dermatologist_conditions_split.csv"
df_filtered = pd.read_csv(df_filtered_path)
df_filtered

,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us,condition_weight
0,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,0.41
1,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,0.41
2,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,0.18
3,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,0.41
4,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10175,983265958160210646,SCIN,1.0.0,2023,AGE_18_TO_29,FEMALE,FST5,0.0,0.0,1.0,...,NaN,NaN,FST5,NaN,NaN,True,True,4.0,5.0,0.50
10176,995820220956352730,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,FST5,0.0,0.0,0.0,...,NaN,NaN,FST4,NaN,NaN,True,True,3.0,2.0,0.33
10177,995820220956352730,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,FST5,0.0,0.0,0.0,...,NaN,NaN,FST4,NaN,NaN,True,True,3.0,2.0,0.33
10178,995820220956352730,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,FST5,0.0,0.0,0.0,...,NaN,NaN,FST4,NaN,NaN,True,True,3.0,2.0,0.33


## Baseline Model

In [120]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [122]:
# Get unique case IDs
case_ids = df_filtered["case_id"].unique()
# Split train and test cases
train_ids, test_ids = train_test_split(case_ids, test_size=0.4, random_state=42)

In [123]:
train_df = df_filtered[df_filtered["case_id"].isin(train_ids)]
test_df = df_filtered[df_filtered["case_id"].isin(test_ids)]

In [126]:
df_train_image_paths = get_all_image_paths(train_df)
df_test_image_paths = get_all_image_paths(train_df)

📸 Found 6255 unique image paths.
📸 Found 6255 unique image paths.
